In [ ]:
# Set Dependencies
import pandas as pd

In [ ]:
# Load the Files for school and students
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [ ]:
# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
merged_school_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
merged_school_data.head()

In [ ]:
# District Summary

In [ ]:
# Compute total unique schools
school_count = merged_school_data["school_name"].nunique()

In [ ]:
# Compute total students
student_count = merged_school_data["Student ID"].nunique()

In [ ]:
# Compute total budget
total_budget = school_data["budget"].sum()

In [ ]:
# Compute average math score
average_math_score = student_data["math_score"].mean()

In [ ]:
# Compute average reading score
average_reading_score = student_data["reading_score"].mean()

In [ ]:
# Compute percentage of students who passed math (greather than or equal to 70)
passing_math_count = merged_school_data[(merged_school_data["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100

In [ ]:
# Compute percentage of students who passed reading (greather than or equal to 70)
passing_reading_count = merged_school_data[(merged_school_data["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100

In [ ]:
# Compute percentage of students who passed math AND reading
passing_math_and_reading_count = merged_school_data[(merged_school_data["math_score"] >= 70) & (merged_school_data["reading_score"] >= 70)
].count()["student_name"]
overall_passing_percentage = passing_math_and_reading_count /  float(student_count) * 100

In [ ]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_summary = pd.DataFrame({"Total Schools"           :[school_count], 
                                 "Total Students"          :[student_count],
                                 "Total Budget"            :[total_budget], 
                                 "Average Math Score"      :[average_math_score],
                                 "Average Reading Score"   :[average_reading_score], 
                                 "% Passing Math"          :passing_math_percentage,
                                 "% Reading"               :passing_reading_percentage, 
                                 "% Overall Passing"       :overall_passing_percentage})

# Apply Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary

In [ ]:
# School Summary

In [ ]:
# Compute the School name and School type
school_types = school_data.set_index(["school_name"])["type"]

In [ ]:
# Compute the total # of students per school
per_school_counts = school_data.set_index(["school_name"])["size"]

In [ ]:
# Calculate the total school budget and per capita spending
per_school_budget = merged_school_data.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

In [ ]:
# Compute average math and reading scores per school
average_math_scores_per_school = merged_school_data.groupby(["school_name"]).mean()["math_score"]
average_reading_scores_per_school = merged_school_data.groupby(["school_name"]).mean()["reading_score"]

In [ ]:
# Compute the # of students passing math and # of students passing reading per school
students_passing_math = merged_school_data[merged_school_data["math_score"] >= 70].groupby("school_name").count()["Student ID"]
students_passing_reading = merged_school_data[merged_school_data["reading_score"] >= 70].groupby("school_name").count()["Student ID"]

In [ ]:
# Compute the % of students passing math and % of students passing reading per school
percent_of_students_passing_math = students_passing_math / per_school_counts * 100
percent_of_students_passing_reading = students_passing_reading / per_school_counts * 100

In [ ]:
# Compute percentage of students who passed math AND reading per school
math_and_reading_passing_scores = merged_school_data[(merged_school_data["reading_score"] >= 70) & (merged_school_data["math_score"] >= 70)]

overall_passing_percentage_per_school = math_and_reading_passing_scores.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

In [ ]:
# Create a DataFrame called `per_school_summary`
per_school_summary = pd.DataFrame({"School Type"             : school_types, 
                                   "Total Students"          : per_school_counts, 
                                   "Total School Budget"     : per_school_budget, 
                                   "Per Student Budget"      : per_school_capita,
                                   "Average Math Score"      : average_math_scores_per_school, 
                                   "Average Reading Score"   : average_reading_scores_per_school,
                                   "% Passing Math"          : percent_of_students_passing_math, 
                                   "% Passing Reading"       : percent_of_students_passing_reading,
                                   "% Overall Passing"       : overall_passing_percentage_per_school})  

# Apply Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display DataFrame
per_school_summary

In [ ]:
#  Highest-Performing Schools (by % Overall Passing)

In [ ]:
# Sort the schools by % Overall Passing in descending order
top_schools = per_school_summary.sort_values("% Overall Passing", ascending=False)

# Display DataFrame (top 5 rows)
top_schools.head(5)

In [ ]:
#  Bottom Performing Schools (By % Overall Passing)

In [ ]:
# Sort the schools by % Overall Passing in ascending  order
bottom_schools = per_school_summary.sort_values("% Overall Passing")

# Display DataFrame (top 5 rows)
bottom_schools.head(5)

In [ ]:
#  Math Scores by Grade

In [ ]:
# Compute the average math score for students of each grade level
ninth_graders = merged_school_data[(merged_school_data["grade"] == "9th")]
tenth_graders = merged_school_data[(merged_school_data["grade"] == "10th")]
eleventh_graders = merged_school_data[(merged_school_data["grade"] == "11th")]
twelfth_graders = merged_school_data[(merged_school_data["grade"] == "12th")]

# Group by "school_name" and take the mean of each
ninth_graders_scores = ninth_graders.groupby("school_name").mean()
tenth_graders_scores = tenth_graders.groupby("school_name").mean()
eleventh_graders_scores = eleventh_graders.groupby("school_name").mean()
twelfth_graders_scores = twelfth_graders.groupby("school_name").mean()

# Select only the math_score
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores.mean()["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame
math_scores_by_grade = pd.DataFrame({"9th"    : ninth_grade_math_scores, 
                                     "10th"   : tenth_grader_math_scores, 
                                     "11th"   : eleventh_grader_math_scores,
                                     "12th"   : twelfth_grader_math_scores})  

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade


In [ ]:
#  Reading Scores by Grade

In [ ]:
# Compute the average reading score for students of each grade level
ninth_graders = merged_school_data[(merged_school_data["grade"] == "9th")]
tenth_graders = merged_school_data[(merged_school_data["grade"] == "10th")]
eleventh_graders = merged_school_data[(merged_school_data["grade"] == "11th")]
twelfth_graders = merged_school_data[(merged_school_data["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby("school_name").mean()
tenth_graders_scores = tenth_graders.groupby("school_name").mean()
eleventh_graders_scores = eleventh_graders.groupby("school_name").mean()
twelfth_graders_scores = twelfth_graders.groupby("school_name").mean()

# Select only the reading_score.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores.mean()["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame
reading_scores_by_grade = pd.DataFrame({"9th"    : ninth_grade_reading_scores, 
                                        "10th"   : tenth_grader_reading_scores, 
                                        "11th"   : eleventh_grader_reading_scores,
                                        "12th"   : twelfth_grader_reading_scores})  

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

In [ ]:
#  Scores by School Spending

In [ ]:
# Establish bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary
school_spending_df = per_school_summary.copy()

In [ ]:
# Categorize spending based on the bins

school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=labels)
school_spending_df

In [ ]:
# Compute averages for the desired columns
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [ ]:
# Assemble into DataFrame
spending_summary = school_spending_df[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading", "% Overall Passing"]]

# Create a dataframe called spending_summary
spending_summary = pd.DataFrame({"Average Math Score"        :spending_math_scores,
                                 "Average Reading Score"     :spending_reading_scores,
                                 "% Passing Math"            :spending_passing_math,
                                 "% Passing Reading"         :spending_passing_reading,
                                 "% Overall Passing"         :overall_passing_spending}) 

# Display Scores grouped by School Spending
spending_summary

In [ ]:
#  Scores by School Size

In [ ]:
# Establish bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize spending based on the bins

per_school_summary["School Size"] = pd.cut(per_school_counts, size_bins, labels=labels)
per_school_summary

In [ ]:
# Compute averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

In [ ]:
# Assemble into DataFrame
size_summary = school_spending_df[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading", "% Overall Passing"]]

# Create a dataframe called spending_summary
size_summary = pd.DataFrame({"Average Math Score"        :size_math_scores,
                             "Average Reading Score"     :size_reading_scores,
                             "% Passing Math"            :size_passing_math,
                             "% Passing Reading"         :size_passing_reading,
                             "% Overall Passing"         :size_overall_passing}) 

# Display Scores grouped by School Spending
size_summary

In [ ]:
#  Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and compute average
type_math_scores = per_school_summary.groupby(["School Type"]).mean()['Average Math Score']
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()['Average Reading Score']
type_passing_math = per_school_summary.groupby(["School Type"]).mean()['% Passing Math']
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()['% Passing Reading']
type_overall_passing = per_school_summary.groupby(["School Type"]).mean()['% Overall Passing']

In [ ]:
# Assemble into DataFrame
type_summary = per_school_summary[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading", "% Overall Passing"]]

# Create a dataframe called type_summary
type_summary = pd.DataFrame({"Average Math Score"            :type_math_scores, 
                             "Average Reading Score"         :type_reading_scores,
                             "% Passing Math"                :type_passing_math,
                             "% Passing Reading"             :type_passing_reading,
                             "% Overall Passing"             :type_overall_passing}) 

# Display Scores grouped by School Type
type_summary